In [ ]:
import pandas as pd
from pandas import read_csv
from datetime import datetime
from pandas import DataFrame
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error

url = 'grafana_data_export.csv'
#df1 = pd.read_csv(url)
# series = read_csv(url, header=0, index_col=0, parse_dates=True, squeeze=True)
series = read_csv(url,header=1,parse_dates=True)
series
# print(series)

In [ ]:
import pandas as pd

df = pd.read_csv('grafana_data_export.csv', header=1, engine='python')

# 將 'TID(converted)' 欄位轉換為時間戳記，以便進行時間的分組和計算
df['TID(converted)'] = pd.to_datetime(df['TID(converted)'], yearfirst=True)

# 將 DataFrame 的索引設置為 'TID(converted)' 欄位的時間戳記
df.index = df['TID(converted)']

# 按照每小時重新取樣，統計每個時間區間內不重複的 MAC 位址數量
half_hourly_count = df['MAC'].resample('30T').nunique()

series=half_hourly_count
print(series)

In [ ]:
from pandas.plotting import lag_plot
# series = read_csv(url, header=0, index_col=0)
lag_plot(series)
plt.show()

In [ ]:
from pandas import concat
# series = read_csv(url, header=0, index_col=0)
values = DataFrame(series.values)
dataframe = concat([values.shift(1), values], axis=1)
dataframe.columns = ['t-1', 't+1']
result = dataframe.corr()
print(result)

In [ ]:
from pandas.plotting import autocorrelation_plot
# series = read_csv(url, header=0, index_col=0)
autocorrelation_plot(series)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
# series = read_csv(url, header=0, index_col=0)
plot_acf(series, lags=10)
plt.show()

In [ ]:
# split dataset
X = series.values
# train, test = X[1:len(X)-48], X[len(X)-48:]

# train-test split for time series
train_size = int(len(series) * 0.8)
test_size = len(series) - train_size
train, test = series.values[:train_size], series.values[train_size:]


from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error
from math import sqrt
# train autoregression
# model = AutoReg(train, lags=500)
model = AutoReg(train, lags=20)
# model = AutoReg(train, lags=3000)
model_fit = model.fit()
print('Coefficients: %s' % model_fit.params)

# make predictions
further_pred_length: int = 50
predictions = model_fit.predict(start=len(train),
                                end=len(train)+len(test)-1+further_pred_length,
                                # dynamic=False,
                                dynamic=True,
                                )
# for i in range(len(predictions)):
	# print('predicted=%f, expected = %d' % (predictions[i], test[i]))
for i in range(len(test)):
  if i % 15 == 0:
    print(f'[{i}] predicted={predictions[i]}, expected = {test[i]}')
rmse = sqrt(mean_squared_error(test, predictions[:len(test)]))
print('Test RMSE: %.3f' % rmse)

In [ ]:
# plot results
fig, ax = plt.subplots(figsize=(10, 4))
fig.suptitle('[AutoRegression] Crowd Forecasting')
ax.set_title(f'RMSE={rmse:.2f}, Length of Future prediced: {further_pred_length}')
ax.set_xlabel('Time points')
ax.set_ylabel('number of people')
ax.plot(test, label='GT')
ax.plot(predictions, color='red', label='predictions')
y_lower, y_upper = ax.get_ybound()
ax.vlines(x=len(test), ymin=y_lower, ymax=y_upper, colors='g', linestyles='dashdot')
ax.legend()
plt.show()

In [ ]:
import mysql.connector

connection = mysql.connector.connect(host='localhost',
                                     port='3306',
                                     user='root',
                                     password='yuting89830') 

cursor = connection.cursor()

#create database
#cursor.execute("CREATE DATABASE `crowd_fore`;")

#get all name of database
# cursor.execute("SHOW DATABASES;")
# records = cursor.fetchall()
# for r in records:
#     print(r)

# use database
cursor.execute("USE `crowd_fore`;")

# create table
#cursor.execute("CREATE TABLE `forecast`(TID VARCHAR(20) , PD INT , GT INT , RMSE DECIMAL(10,2) , TP INT)")

# # 執行 SQL 查詢
# cursor.execute("SELECT * FROM forecast;")  # 假設你的資料表名稱是 forecast

# # 取得查詢結果
# result = cursor.fetchall()


In [ ]:
#清空資料表內容但保留格式
#cursor.execute("TRUNCATE TABLE `forecast`;")
#修改資料表格式
#cursor.execute("ALTER TABLE `forecast` MODIFY COLUMN GT INT;")

In [ ]:
# 假設預測結果從測試集結束後的一個時間點開始，每30分鐘預測一次
start_time = series.index[train_size]  # 取得測試集結束後的一個時間點
time_interval = pd.Timedelta(minutes=30)  # 時間間隔為30分鐘
predicted_time_points = [start_time + i * time_interval for i in range(len(predictions))]

In [ ]:
from datetime import datetime, timedelta

# 假設每30分鐘蒐集一次數據
sampling_interval = timedelta(minutes=30)

# 將TID、預測結果、真實值、RMSE 和 time point 存入資料庫
for i in range(len(predictions)):
    TID_index = train_size + i  # 在 series 中的索引位置
    if TID_index < len(series):
        TID = series.index[TID_index]  # 取得對應到的真實時間
    else:
        # 超出 series 的範圍，每輪增加半小時
        last_time = series.index[-1]
        TID = last_time + (i - len(series) + train_size) * sampling_interval

    PD = float(predictions[i])  # 將 numpy.float64 轉換成 float
    GT = float(test[i]) if i < len(test) else None  # 將 numpy.float64 轉換成 float
    TP = i  # time point
    RMSE = float(sqrt(mean_squared_error(test, predictions[:len(test)])))  # 將 numpy.float64 轉換成 float

    print(TID,PD,GT,TP,RMSE)

In [ ]:
from datetime import datetime, timedelta

# 假設每30分鐘蒐集一次數據
sampling_interval = timedelta(minutes=30)

# 將TID、預測結果、真實值、RMSE 和 time point 存入資料庫
for i in range(len(predictions)):
    TID_index = train_size + i  # 在 series 中的索引位置
    if TID_index < len(series):
        TID = series.index[TID_index]  # 取得對應到的真實時間
    else:
        # 超出 series 的範圍，每輪增加半小時
        last_time = series.index[-1]
        TID = last_time + (i - len(series) + train_size) * sampling_interval

    PD = float(predictions[i])  # 將 numpy.float64 轉換成 float
    GT = float(test[i]) if i < len(test) else None  # 將 numpy.float64 轉換成 float
    TP = i  # time point
    RMSE = float(sqrt(mean_squared_error(test, predictions[:len(test)])))  # 將 numpy.float64 轉換成 float

    print(TID,PD,GT,TP,RMSE)

    # 建立插入資料的 SQL 查詢
    sql = "INSERT INTO forecast (TID, PD, GT, RMSE, TP) VALUES (%s, %s, %s, %s, %s);"
    values = (TID, PD, GT, RMSE, TP)

    # 執行插入資料
    cursor.execute(sql, values)
    connection.commit()

In [ ]:
# 執行 SQL 查詢來取得資料表的標題
cursor.execute("DESCRIBE forecast;")  # 假設你的資料表名稱是 forecast

# 取得查詢結果，即欄位名稱
columns = [column[0] for column in cursor.fetchall()]

# 印出標題
print("資料表標題：")
print(columns)

# 執行 SQL 查詢來取得資料表內容
cursor.execute("SELECT * FROM forecast;")

# 取得查詢結果，即資料表內容
data = cursor.fetchall()

# 印出內容
print("資料表內容：")
for row in data:
    print(row)
    
#close the database and disconnection with mysql
cursor.close()
connection.close()